<a href="https://colab.research.google.com/github/EntropyP/News_Update/blob/main/News_Update_Main_Program.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# News Update acquisition : Main Program

## Set up

In [84]:
! pip install newsapi-python

In [85]:
# import libraries

from newsapi import NewsApiClient
import datetime as datetime
import pandas as pd

In [86]:
# input api_key (Request from https://newsapi.org/)

newsapi = NewsApiClient(api_key = 'a51013f42c6d46e3b3fa974d7363d177')

## Input key search and maximum page search

In [87]:
key_search = ['"DOW chemical"','"carbon capture and storage"']
page_search = 100

## Searching Process

In [88]:
#Create final dataframe which will collect all news in the focus area according to key_search

df_final = pd.DataFrame()

#Search data from each key_search

for key in key_search:
  try:
    data = newsapi.get_everything(q = key, language = 'en', page_size= page_search, sort_by='relevancy')
    articles = data['articles']
    df_news = pd.DataFrame(articles)
    df_news['key'] = key
    df_final = df_final.append(df_news,ignore_index=True)
  except:
    print('There is no search result for "', key ,'"')

#The final dataframe which was classified by topic

df_final['source_name'] = df_final['source'].apply(pd.Series)['name']
df_final

,source,author,title,description,url,urlToImage,publishedAt,content,key,source_name
0,"{'id': 'al-jazeera-english', 'name': 'Al Jazee...",Al Jazeera,Lucid Motors plans to build EV factory in Saud...,Chairman told Bloomberg News that the EV maker...,https://www.aljazeera.com/economy/2022/1/12/bb...,https://www.aljazeera.com/wp-content/uploads/2...,2022-01-12T13:55:50Z,Lucid Group Inc. intends to build an electric-...,"""DOW chemical""",Al Jazeera English
1,"{'id': None, 'name': 'Yahoo Entertainment'}",GRAPHICAL ANALYTICS PRIVATE LIMITED,"Biocides Market Size & Share | North America, ...","Major biocides market players include DuPont, ...",https://finance.yahoo.com/news/biocides-market...,https://s.yimg.com/uu/api/res/1.2/0vjWRr0ZfZNd...,2022-01-11T10:00:00Z,"Major biocides market players include DuPont, ...","""DOW chemical""",Yahoo Entertainment
2,"{'id': None, 'name': 'CNA'}",None,China to impose anti-dumping tariffs on US che...,BEIJING : China's commerce ministry said o...,https://www.channelnewsasia.com/business/china...,https://onecms-res.cloudinary.com/image/upload...,2022-01-10T09:34:23Z,BEIJING : China's commerce ministry said on Mo...,"""DOW chemical""",CNA
3,"{'id': None, 'name': 'Yahoo Entertainment'}",PR Newswire,Dow named one of America's Most JUST Companies...,Dow (NYSE: DOW) announced today that for the t...,https://finance.yahoo.com/news/dow-named-one-a...,https://s.yimg.com/uu/api/res/1.2/.yQJ3zsMBtTQ...,2022-01-11T14:47:00Z,Earns top spot in Chemicals sector and #1 rank...,"""DOW chemical""",Yahoo Entertainment
4,"{'id': None, 'name': 'Hyperallergic'}",Seph Rodney,Saying Maybe to the Yes Men,As much as I appreciate the collective’s cultu...,http://hyperallergic.com/704394/saying-maybe-t...,https://i0.wp.com/hyperallergic-newspack.s3.am...,2022-01-18T22:16:00Z,"Years ago, when I was a graduate student in th...","""DOW chemical""",Hyperallergic
...,...,...,...,...,...,...,...,...,...,...
187,"{'id': None, 'name': 'Forbes'}","Felicia Jackson, Contributor, \n Felicia Jacks...",Energy Transition Will Drive Geopolitical Chan...,Calls continue to triple renewable energy capa...,https://www.forbes.com/sites/feliciajackson/20...,https://thumbor.forbes.com/thumbor/fit-in/1200...,2022-01-17T15:57:04Z,"LONDON, ENGLAND - OCTOBER 04: Francesco La Cam...","""carbon capture and storage""",Forbes
188,"{'id': None, 'name': 'Business Wire'}",None,Daniel Schrag to Join Climate Vault’s Technolo...,"CHICAGO--(BUSINESS WIRE)--Climate Vault, an aw...",https://www.businesswire.com/news/home/2022011...,https://mms.businesswire.com/media/20220110005...,2022-01-10T16:54:17Z,"CHICAGO--(BUSINESS WIRE)--Climate Vault, an aw...","""carbon capture and storage""",Business Wire
189,"{'id': None, 'name': 'Zerohedge.com'}",Mike Rivero,US Gov't Spent $1.1B On Failed Carbon Capture ...,The U.S. Department of Energy has spent $1.1 b...,https://www.zerohedge.com/energy/us-govt-spent...,https://assets.zerohedge.com/s3fs-public/style...,2022-01-12T20:43:06Z,Authored by Tsvetana Paraskova via OilPrice.co...,"""carbon capture and storage""",Zerohedge.com
190,"{'id': None, 'name': 'Wattsupwiththat.com'}",Charles Rotter,Decarbonisation tech instantly converts CO2 to...,"As the bubbles move through the liquid metal, ...",https://wattsupwiththat.com/2022/01/20/decarbo...,https://149366104.v2.pressablecdn.com/wp-conte...,2022-01-20T18:00:00Z,New tech offers pathway for instantly converti...,"""carbon capture and storage""",Wattsupwiththat.com


In [89]:
#Save to excel
file_name = input('Please enter file name: ')
df_final.to_excel(file_name + '.xlsx')

Please enter file name: latest


# Access to the content

## Set up

In [90]:
import bs4
import requests

headers = {
    'User-agent':
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.102 Safari/537.36 Edge/18.19582"
}

## Create function for web scrapping

In [159]:
#reuters
def reuters():
  content = soup.findAll('p', class_= 'Text__text___3eVx1j Text__dark-grey___AS2I_p Text__regular___Bh17t- Text__large___1i0u1F Body__base___25kqPt Body__large_body___3g04wK ArticleBody__element___3UrnEs')
  content_final = ''
  for con in content:
    content_final += con.text
  return content_final

#yahoo
def yahoo():
  content = soup.findAll('p')
  content_final = ''
  for con in content:
    if '<p class=' in str(con):
      continue
    else:
      content_final += con.text
  return content_final

## Create function for text summatization

In [150]:
from gensim.summarization.summarizer import summarize


# Content extraction

In [161]:
df_extract = df_final.copy(deep=True)
df_extract = df_extract[['title','url','publishedAt','key','source_name']]
df_extract['Summary'] = None
df_extract['Full_Text'] = None
df_extract

,title,url,publishedAt,key,source_name,Summary,Full_Text
0,Lucid Motors plans to build EV factory in Saud...,https://www.aljazeera.com/economy/2022/1/12/bb...,2022-01-12T13:55:50Z,"""DOW chemical""",Al Jazeera English,None,None
1,"Biocides Market Size & Share | North America, ...",https://finance.yahoo.com/news/biocides-market...,2022-01-11T10:00:00Z,"""DOW chemical""",Yahoo Entertainment,None,None
2,China to impose anti-dumping tariffs on US che...,https://www.channelnewsasia.com/business/china...,2022-01-10T09:34:23Z,"""DOW chemical""",CNA,None,None
3,Dow named one of America's Most JUST Companies...,https://finance.yahoo.com/news/dow-named-one-a...,2022-01-11T14:47:00Z,"""DOW chemical""",Yahoo Entertainment,None,None
4,Saying Maybe to the Yes Men,http://hyperallergic.com/704394/saying-maybe-t...,2022-01-18T22:16:00Z,"""DOW chemical""",Hyperallergic,None,None
...,...,...,...,...,...,...,...
187,Energy Transition Will Drive Geopolitical Chan...,https://www.forbes.com/sites/feliciajackson/20...,2022-01-17T15:57:04Z,"""carbon capture and storage""",Forbes,None,None
188,Daniel Schrag to Join Climate Vault’s Technolo...,https://www.businesswire.com/news/home/2022011...,2022-01-10T16:54:17Z,"""carbon capture and storage""",Business Wire,None,None
189,US Gov't Spent $1.1B On Failed Carbon Capture ...,https://www.zerohedge.com/energy/us-govt-spent...,2022-01-12T20:43:06Z,"""carbon capture and storage""",Zerohedge.com,None,None
190,Decarbonisation tech instantly converts CO2 to...,https://wattsupwiththat.com/2022/01/20/decarbo...,2022-01-20T18:00:00Z,"""carbon capture and storage""",Wattsupwiththat.com,None,None


In [162]:
for row, column in df_extract.iterrows():
  if column['source_name'] == 'Reuters':
    ori_link = column['url']
    link = requests.get(ori_link,headers=headers)
    soup = bs4.BeautifulSoup(link.text,'lxml')
    try:
      text = reuters()
      df_extract.loc[row, ['Summary']] = summarize(text, ratio=0.3)
      df_extract.loc[row, ['Full_Text']] = text
    except:
      pass
  elif column['source_name'] == 'Yahoo Entertainment':
    ori_link = column['url']
    link = requests.get(ori_link,headers=headers)
    soup = bs4.BeautifulSoup(link.text,'lxml')
    try:
      text = yahoo()
      df_extract.loc[row, ['Summary']] = summarize(text, ratio=0.3)
      df_extract.loc[row, ['Full_Text']] = text
    except:
      pass

In [164]:
# for row, column in df_extract.iterrows():
#   if column['source_name'] == 'Reuters':
#     ori_link = column['url']
#     link = requests.get(ori_link,headers=headers)
#     soup = bs4.BeautifulSoup(link.text,'lxml')
#     try:
#       text = reuters()
#       print(row,column['title'])
#       print(text)
#       print()
#     except:
#       pass
#   elif column['source_name'] == 'Yahoo Entertainment':
#     ori_link = column['url']
#     link = requests.get(ori_link,headers=headers)
#     soup = bs4.BeautifulSoup(link.text,'lxml')
#     try:
#       print(row,column['title'])
#       content = soup.findAll('p')
#       text = ''
#       for con in content:
#         if '<p class=' in str(con):
#           pass
#         else:
#           text += con.text
#       print(text)
#       print()
#     except:
#       pass

In [165]:
#Save to excel
df_extract = df_extract.dropna()
df_extract.to_excel(file_name + '_summary.xlsx')